### <b>4. Classification par réseaux de neurones convolutifs</b>
## <b>Partie 1 : Petits réseaux de neurones personnalisés</b>

### <b>Exercice 4.1 : Réseau de neurones convolutifs avec données GTSRB (Keras)</b>

Développer et tester un réseau de neurones convolutifs en Keras, permettant d'obtenir une <b>précision (accuracy) >=99,5%</b> aves les données GTSRB. Le modèle devra comporter <b>moins de 300000 paramètres</b>.

On pourra reprendre dans un premier temps la structure générale du réseau de neurones déjà utilisé avec MNIST, mais il faudra ensuite l'adapter pour les images couleurs et pour améliorer sa précision.  
On prendra une taille de 48x48 pour les images.

On peut reprendre l'exemple complet de classification des données GTSRB avec un MLP.
Pour l'amélioration de la précision, on pourra jouer sur :
- le nombre de couches (convolutives et complètement connectées)
- le nombre de feature-maps dans les couches convolutives
- le pooling
- le dropout
- la taille des batchs
- l'augmentation du dataset
- la batch-normalisation
- ...

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import glob
import cv2
import numpy as np

# Configuration de base
input_shape = (48, 48, 3)  # Images 48x48 en couleur
num_classes = 43  # Nombre de classes dans GTSRB
data_path = '/home/jovyan/iadatasets/GTSRB/Final_Training/Images/'  # Chemin vers les données

# Chargement et préparation des données
def load_GTSRB():
    print("Chargement des données...")
    data, labels = [], []
    for i in range(num_classes):
        image_path = os.path.join(data_path, format(i, '05d'))
        print(f"Chargement du répertoire {image_path}")
        for img_path in glob.glob(image_path + '/*.ppm'):
            image = cv2.imread(img_path)
            image = cv2.resize(image, (48, 48))  # Redimensionnement à 48x48
            data.append(image)
            labels.append(i)
    print('Données chargées.')
    return np.array(data), np.array(labels)

X, y = load_GTSRB()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Normalisation des données
X_train, X_test = X_train / 255.0, X_test / 255.0

# Création du modèle CNN
model = Sequential()

# Couche convolutive 1
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Couche convolutive 2
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# Couche d'aplatissement
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Couche de sortie
model.add(Dense(num_classes, activation='softmax'))

# Compilation du modèle
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, validation_data=(X_test, y_test)) # 30 epochs suffisent pour atteindre la précision

model.summary()

# Évaluation du modèle sur les données de test
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Précision sur l\'ensemble de test : {score[1] * 100:.2f}%')

# Affichage des courbes de précision
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Précision entraînement')
plt.plot(history.history['val_accuracy'], label='Précision validation')
plt.xlabel('Époques')
plt.ylabel('Précision')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Perte entraînement')
plt.plot(history.history['val_loss'], label='Perte validation')
plt.xlabel('Époques')
plt.ylabel('Perte')
plt.legend()

plt.show()

##### <b>Commentaires</b>

Nous avons ajouté 2 couches de convolution, possédant un nombre croissant de feature maps (64 puis 128), ce qui nous permet de ne pas dépasser 300k paramètres. Les couches sont suivies de Batch Normalization et de MaxPooling2D pour stabiliser l'apprentissage et réduire la taille des cartes de caractéristiques. Nous avons commencé l'apprentissage avec 50 epochs, ce qui nous permet d'atteindre largement l'objectif mais qui est trop long. Nous avons donc reéssayé avec 20 epochs, ce qui est suffisant pour atteindre la précision demandée.

### <b>Exercice 4.2 : Réseau de neurones convolutifs avec données GTSRB (Pytorch)</b>

Idem exercice 4.1 mais avec Pytorch.  

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchsummary import summary
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import glob

# Paramètres de configuration
input_shape = (48, 48, 3)  # Images 48x48 en couleur
num_classes = 43  # Nombre de classes dans GTSRB
data_path = '/home/jovyan/iadatasets/GTSRB/Final_Training/Images/'

# Chargement des données et définition des transformations
class GTSRBDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Chargement et préparation des données
def load_GTSRB():
    data, labels = [], []
    for i in range(num_classes):
        image_path = os.path.join(data_path, format(i, '05d'))
        for img_path in glob.glob(image_path + '/*.ppm'):
            image = cv2.imread(img_path)
            image = cv2.resize(image, (48, 48))  # Redimensionnement à 48x48
            data.append(image)
            labels.append(i)
    return np.array(data), np.array(labels)

# Transformations avec augmentation de données
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((48, 48)),
    transforms.RandomRotation(15),  # Rotation
    transforms.RandomHorizontalFlip(),  # Flip horizontal
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Chargement des données
X, y = load_GTSRB()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Création des jeux de données PyTorch
train_dataset = GTSRBDataset(X_train, y_train, transform=transform)
test_dataset = GTSRBDataset(X_test, y_test, transform=transform)

# Création des DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Création du modèle CNN
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),
            nn.Dropout(0.3),
            
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),
            nn.Dropout(0.3)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 6 * 6, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = CNNModel(num_classes=num_classes)

# Critère et Optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Fonction d'entraînement avec suivi des pertes et précisions
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    train_losses, train_accuracies = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_accuracy)

        print(f'Époque {epoch+1}/{num_epochs}, Perte: {epoch_loss:.4f}, Précision: {epoch_accuracy:.2f}%')
    
    return train_losses, train_accuracies

# Fonction évaluation
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = 100 * correct / total
    print(f'Précision sur l\'ensemble de test : {test_accuracy:.2f}%')
    return test_accuracy

# Entraînement du modèle
train_losses, train_accuracies = train_model(model, train_loader, criterion, optimizer, num_epochs=30)

# Évaluation du modèle
test_accuracy = evaluate_model(model, test_loader)

# Affichage des courbes de précision
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_accuracies, label='Précision entraînement')
plt.xlabel('Époques')
plt.ylabel('Précision')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_losses, label='Perte entraînement')
plt.xlabel('Époques')
plt.ylabel('Perte')
plt.legend()

plt.show()

Époque 1/30, Perte: 1.3738, Précision: 61.90%
Époque 2/30, Perte: 0.4533, Précision: 86.24%
Époque 3/30, Perte: 0.2568, Précision: 91.95%
Époque 4/30, Perte: 0.1975, Précision: 93.80%
Époque 5/30, Perte: 0.1613, Précision: 94.88%
Époque 6/30, Perte: 0.1500, Précision: 95.18%
Époque 7/30, Perte: 0.1319, Précision: 95.79%
Époque 8/30, Perte: 0.1217, Précision: 96.05%
Époque 9/30, Perte: 0.1146, Précision: 96.27%
Époque 10/30, Perte: 0.0993, Précision: 96.70%
Époque 11/30, Perte: 0.0898, Précision: 97.27%
Époque 12/30, Perte: 0.0923, Précision: 96.88%
Époque 13/30, Perte: 0.0852, Précision: 97.29%
Époque 14/30, Perte: 0.0857, Précision: 97.06%
Époque 15/30, Perte: 0.0751, Précision: 97.47%
Époque 16/30, Perte: 0.0742, Précision: 97.55%
Époque 17/30, Perte: 0.0695, Précision: 97.65%
Époque 18/30, Perte: 0.0608, Précision: 98.04%
Époque 19/30, Perte: 0.0652, Précision: 97.95%
Époque 20/30, Perte: 0.0618, Précision: 97.98%
Époque 21/30, Perte: 0.0596, Précision: 97.98%
Époque 22/30, Perte: 0

##### <b>Commentaires</b>

Nous avons adapté notre réseau de neurones en PyTorch. Nous avons d'abord préparé les données avec des transformations pour augmenter la diversité des images, puis avons entraîné notre modèle avec 30 époques en suivant les pertes et la précision à chaque étape. Ensuite, nous avons évalué la performance de notre modèle sur l'ensemble de test, en affichant les courbes de précision et de perte pour observer l'évolution de l'apprentissage et des résultats. Le modèle respecte bien la limite de 300 000 paramètres.

### <b>Exercice 4.3 : Chargement d'un modèle généré précédemment et utilisation en inférence</b>

Appliquer le modèle en <b>inférence</b> sur au moins une image du dataset de test (il faudra pour cela au préalable convertir cette image au format tenseur de torch puis l'appliquer en entrée du modèle).  
Mesurer la <b>durée</b> d'inférence (en l'appliquant sur la même image et pour un grand nombre d'itérations, et en affichant cette durée à chaque fois).
Afficher sur l'image si la réponse est correcte ou non.  

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from PIL import Image

# Transformation des images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Fonction pour charger et transformer une image pour l'inférence
def transform_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (48, 48))  # Redimensionner en 48x48
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Conversion en RGB
    image = Image.fromarray(image)  # Conversion en image PIL
    image = transform(image).unsqueeze(0)  # Ajout de la dimension du batch
    return image

# Charger le modèle sauvegardé
model_path = "/home/jovyan/TP1/modele/modele.pth"  # Chemin vers le modèle sauvegardé
model = torch.load(model_path)
model.eval()

# Déplacement du modèle sur le GPU si disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Charger une image de test
test_image_path = '/home/jovyan/iadatasets/GTSRB/Final_Training/Images/00000/00000_00000.ppm'
true_label = 0  # Classe réelle de l'image test
image_tensor = transform_image(test_image_path)

# Déplacement de l'image sur le GPU si disponible
image_tensor = image_tensor.to(device)

# Inférence sur plusieurs itérations
num_iterations = 10
durations = []
correct_predictions = 0

for _ in range(num_iterations):
    with torch.no_grad():
        start_time = time.time()
        output = model(image_tensor)
        _, predicted_class = torch.max(output, 1)
        end_time = time.time()

        # Enregistrer la durée d'inférence
        durations.append(end_time - start_time)

        # Vérifier si la prédiction est correcte
        if predicted_class.item() == true_label:
            correct_predictions += 1

# Durée moyenne d'inférence
average_inference_time = sum(durations) / num_iterations
print(f"Durée moyenne d'inférence : {average_inference_time:.4f} secondes")
print(f"Prédictions correctes : {correct_predictions}/{num_iterations}")

# Résultat de la prédiction pour affichage
result_text = "Prédiction correcte !" if predicted_class.item() == true_label else f"Prédiction incorrecte. Classe prédite : {predicted_class.item()}"

# Afficher l'image avec le résultat
image = cv2.imread(test_image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.title(f"{result_text} (Classe réelle : {true_label})")
plt.axis('off')
plt.show()

##### <b>Commentaires</b>

Nous avons repris le modèle de l'exercice précédent que nous avons appliqué en inférence sur la première image du dataset.